In [1]:
%load_ext autoreload
%autoreload 2
%load_ext jupyter_black

In [2]:
from evallm.utils.bootstrap import boostrap_mean

from evallm.experiments.evaluate_tokenization_regexp_non_icl import *
import re

In [3]:
summary(*evaluate_model_regexp_matching_multiple("gpt-4o-2024-05-13", 100))

Percent positive: 52%
Percent correct : 100%
